<a href="https://colab.research.google.com/github/cateto/python4NLP/blob/main/kobert/news_classifications_pytorch_kobert0728.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
# 변경: 최신 버전으로 설치하면 "Input: must be Tensor, not str" 라는 에러 발생
!pip install transformers==3
!pip install torch

In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-lco_sve2
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-lco_sve2


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [6]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
##GPU 사용 시
device = torch.device("cuda:0")

In [8]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
dataset = pd.read_csv('/content/drive/MyDrive/rsn_nlp_project/sample_0728.csv')

#라벨 데이터 category 정수로 변환

encoder = LabelEncoder()
encoder.fit(dataset['category'])
dataset['category'] = encoder.transform(dataset['category'])

# possible_labels = dataset['category'].unique()
# label_dict = {}
# for index, possible_label in enumerate(possible_labels):
#     label_dict[possible_label] = index

# dataset['category'] = dataset['category'].replace(label_dict)

#이어서 contents 전처리(정제)
dataset['contents'] = dataset['contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")
dataset['contents'] = dataset['contents'].str.replace("[ ]{2,}"," ")

dataset.head()

,id,contents,category
0,NIRW1900000029.1,부산 기관장들 국가사업 차질 없이 이뤄내겠다 부산 는 서병수 부산시장을 비롯한 부...,4
1,NIRW1900000029.2,새해 해돋이 명소 등 트래픽 급증 이통 사 비상태세 이상 새해를 맞이해 통화량이 ...,0
2,NIRW1900000029.3,조기 대선의 해 향배 가를 대 변수 는 년 정유년 대통령 선거의 해가 시작됐다 이...,8
3,NIRW1900000029.4,달라지는 소비자제도 어린이운송차 속도제한장치 의무화 올해부터는 어린이 운송용 차량...,3
4,NIRW1900000029.5,광주 간 전화 건 대선주자들의 새해 첫날 지난 한해 정치권 비리로 너무 속상한 시...,8


In [10]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: 'IT/과학',
 1: '경제',
 2: '문화',
 3: '미용/건강',
 4: '사회',
 5: '생활',
 6: '스포츠',
 7: '연예',
 8: '정치'}

In [11]:
# test data / train data 나누기

train, test = train_test_split(dataset, test_size=0.2, random_state=42)
print("훈련용 data 개수:", len(train))
print("테스트용 data 개수:", len(test))
train.to_csv('/content/drive/MyDrive/rsn_nlp_project/sample_train.txt', sep = '\t' , index = False)
test.to_csv('/content/drive/MyDrive/rsn_nlp_project/sample_test.txt', sep = '\t' , index = False)

훈련용 data 개수: 102833
테스트용 data 개수: 25709


재시도

In [12]:
dataset_train = nlp.data.TSVDataset("/content/drive/MyDrive/rsn_nlp_project/sample_train.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("/content/drive/MyDrive/rsn_nlp_project/sample_test.txt", field_indices=[1,2], num_discard_samples=1)

In [13]:
dataset_train[:5]

[[' 면 제주 주년 광화문 문화제 일 열려 판에 현기영 김석범 대담 업뎃 광화문 광장에서 제주 주년 국민문화제 열린다 부 잠들지 않는 노래 부 평화콘서트 진행 안치환과 자유 전인권 밴드 등 출연 에 대한 미국 책임 묻는 기자회견 예정 일 오후에는 소설가 김석범 현기영 대담 제주 주년을 맞아 주말 오후 서울 광화문 광장에서 국민문화제가 열린다 제주 제 주년 범국민위원회 범국민위 는 일 오후 시 분부터 서울 광화문 북광장에서 제주 항쟁 주년 광화문 국민문화제 를 연다고 일 밝혔다 이번 국민문화제는 희생자들 넋을 기리고 변방 에서 일어났던 을 서울에서 알리기 위한 것이다 이날 무대는 년 끝나지 않는 노래 를 주제로 부 잠들지 않는 노래 와 부 평화콘서트 로 꾸며진다 부에서는 제주어로 노래를 부르는 밴드 사우스카니발의 무대를 시작으로 이번 무대를 위해 만든 프로젝트 밴드 의 아픔을 보여주는 마임 일어나요 할망 등 다채로운 공연이 꾸려진다 또 제주의 민중가수 최상돈이 평화합창단과 함께 애기 동백꽃의 노래 와 잠들지 않는 남도 를 함께 부른다 또 명으로 구성된 프로젝트 밴드는 항쟁의 역사를 부른다 는 주제로 무대에 오르고 극단 경험과 상상은 의 도화선이 된 년 절 기념대회가 열린 제주시 관덕정 광장을 재현하는 무대를 만든다 이어 부에는 안치환과 자유 멜로망스 전인권 밴드가 출연한다 대학 재학 시절 을 소재로 한 이산하의 장편 서사시 한라산 을 읽고 잠들지 않는 남도 를 작사 작곡한 가수 안치환은 이번 콘서트에서 제주도민의 슬픔을 담담하게 표현한 신곡 월 동백 을 선보인다 청소년들의 관심을 불러일으키기 위해 인기그룹 멜로망스도 무대에 오른다 부는 년 겨울 광화문광장에서 열린 촛불문화제에서 화제를 모은 전인권 밴드가 마무리한다 식전행사로는 정오부터 제주와 서울의 인디밴드 공연 혼디 부르게 바당의 노래 가 진행된다 마로 섬 플레이버 어쩌다밴드 디오디오 묘한 정흠밴드 극도 레이지본 씨없는수박김대중 호선 버터플라이 등이 출연한다 이날 광화문광장에는 과 제주의 전통문화를 알리는 

In [14]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [15]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [16]:
## Setting parameters
max_len = 163 # max_length 512, but Considering Computing Resource => set 163
batch_size = 32 # GPU CUDA problem
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [17]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [18]:
data_train[0]

(array([   2, 2029, 4154, 4213, 5712, 1096, 7941, 6234, 2132, 7234, 3803,
        3358, 6060, 4805, 6896, 5049, 5561, 6951, 1316, 6557, 6333, 1633,
        5798, 3260,    0, 1096, 7941, 6234, 1096, 7184, 4154, 4213, 5712,
        1144, 6238, 7234, 3365, 2423, 3945, 5931, 7318, 3151, 1479, 2423,
        4851, 7540, 4370, 3135, 7483, 7946, 5468, 3920, 4012, 7119, 5524,
         517, 6317, 1815, 4579,  517, 6896, 1682, 2150, 4458, 2134, 5760,
         517, 5580, 7953, 5414, 3413, 3803, 3434, 6900, 2835, 5330, 1316,
        6557, 6333, 5049, 5561, 6951, 1633, 5798, 4154, 4213, 5712, 7088,
        1974, 4223, 3434, 2726, 1096, 7941, 6234, 1096, 7184, 1144, 6238,
        7234, 5330, 3365, 4154, 4128, 4213, 5712, 2318, 5507, 7054, 2318,
        5507, 7044,  517, 5760, 3803, 3434, 2959, 2468, 6410, 2726, 1096,
        7941, 6234, 2462, 5481, 7184, 4154, 4992, 7198, 4213, 5712, 1096,
        7941, 6234, 1144, 6238, 7234,  517, 6116, 3332, 5782, 5439, 3803,
        2261, 3697, 1144, 6238, 7234, 

In [19]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [20]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                #  num_classes=2,
                 num_classes=9,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [21]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [22]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [23]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [24]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [25]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [26]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.258406162261963 train acc 0.09375
epoch 1 batch id 201 loss 1.5925551652908325 train acc 0.23041044776119404
epoch 1 batch id 401 loss 0.5791765451431274 train acc 0.45183915211970077
epoch 1 batch id 601 loss 0.7329061627388 train acc 0.5754471713810316
epoch 1 batch id 801 loss 0.4771496057510376 train acc 0.6419319600499376
epoch 1 batch id 1001 loss 0.3794427514076233 train acc 0.6816933066933067
epoch 1 batch id 1201 loss 0.5340436697006226 train acc 0.7111001248959201
epoch 1 batch id 1401 loss 0.38508814573287964 train acc 0.7315310492505354
epoch 1 batch id 1601 loss 0.5686830878257751 train acc 0.7470331043098064
epoch 1 batch id 1801 loss 0.3707597553730011 train acc 0.7590574680732927


In [ ]:
# 테스트 문장 예측

test_sentence = '태권도 대표팀이 대회 마지막 날 종주국의 면모를 보였다. 비록 ‘노골드’라는 아쉬운 성적을 남겼지만 경기 종료 1초 전 ‘버저비터 발차기’로 태권도의 재미를 높였고, 암을 극복하고 메달을 따내는 ‘인간 승리’의 드라마도 만들었다. 이다빈(25·서울시청)은 27일 일본 지바 마쿠하리 메세 A홀에서 열린 2020 도쿄 올림픽 태권도 여자 67㎏초과급 준결승에서 ‘버저비터 발차기’를 선보였다. 세계랭킹 1위 비안카 워크던(영국)을 맞아 22-24로 끌려가던 이다빈은 경기 종료 직전 왼발을 들어 워크던의 얼굴에 꽂아 넣으며 3점을 따내 극적인 역전승을 일궈냈다. 대회 내내 수비 위주의 경기가 반복되며 ‘발펜싱’이라는 오명을 얻었던 태권도가 극적이고 재밌는 경기라는 점을 이다빈이 준결승에서 제대로 보여줬다.'
test_label = '6'
temp_set = []
temp_set.append(test_sentence)
temp_set.append(test_label)
test_set = []
test_set.append(temp_set)

In [ ]:
test_set

In [ ]:
test_set = BERTDataset(test_set, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=9)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)

  model.eval() # 평가 모드로 변경
      
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)
      valid_length= valid_length
      label = label.long().to(device)
      out = model(token_ids, valid_length, segment_ids)
      test_acc += calc_accuracy(out, label)
  print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))